In [2]:
!nvidia-smi

Fri Jun  7 12:54:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:03:00.0 Off |                  N/A |
|  0%   50C    P5    16W / 240W |      0MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:82:00.0 Off |                  N/A |
|  0%   45C    P5    18W / 210W |      0MiB /  8119MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import itertools
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_hub as hub
import re

from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.layers import Layer
from keras.utils import Sequence
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, Flatten, GlobalMaxPool1D, LSTM
from keras.layers import Input, MaxPooling1D, GlobalAveragePooling1D, multiply, concatenate, Reshape
from keras.layers import Bidirectional
from keras.optimizers import Adam
import keras.initializers as initializers
import keras.regularizers as regularizers
import keras.constraints as constraints


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle
import pickle

sess = tf.Session()


W0607 12:54:57.542215 140315790804800 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [5]:
max_seq_length = 256
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
elmo_path = "https://tfhub.dev/google/elmo/2"

# Load data

In [6]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
        fname="aclImdb.tar.gz", 
        origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
        extract=True)

    train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

    return train_df, test_df

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()

,sentence,sentiment,polarity
0,I watched this film for 45 minutes and counted...,1,0
1,I agree with one of the other comment writers ...,3,0
2,"Contrary to what those who hate Christianity, ...",10,1
3,"OK so there's nudity, but hey, there's free po...",2,0
4,"This film had no huge stars in it, but did hav...",4,0


# ELMO pipeline

## Elmo Tokenizer

In [7]:
from elmo_tokenizer import ELMO_tokenizer

In [8]:
tokenizer = ELMO_tokenizer(max_seq_length)

In [9]:
train_tokens = tokenizer.predict(train_df['sentence'].tolist())
train_label = train_df['polarity'].tolist()

test_tokens = tokenizer.predict(test_df['sentence'].tolist())
test_label = test_df['polarity'].tolist()

## Elmo Model

In [12]:
from elmo_layer import ElmoLayer

In [11]:
def build_model_elmo(max_seq_length): 
    
    input_tokens = Input(shape=(max_seq_length,),dtype="string")
    bert_output = ElmoLayer(trainable=True, tf_hub = elmo_path, output_representation='default')(input_tokens)
    
    dense = Dense(256, activation='relu')(bert_output)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=input_tokens, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [13]:
elmo_model = build_model_elmo(max_seq_length)
initialize_vars(sess)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
elmo_layer_1 (ElmoLayer)     (None, 1024)              93600852  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 93,863,509
Trainable params: 262,661
Non-trainable params: 93,600,848
_________________________________________________________________


In [33]:
elmo_model.trainable_weights

[<tf.Variable 'elmo_layer_1_module/aggregation/weights:0' shape=(3,) dtype=float32>,
 <tf.Variable 'elmo_layer_1_module/aggregation/scaling:0' shape=() dtype=float32>,
 <tf.Variable 'dense_1/kernel:0' shape=(1024, 256) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]

In [15]:
elmo_model.fit(
    train_tokens,
    train_label,
    validation_data=(test_tokens, test_label),
    epochs=1,
    verbose = 1,
    batch_size=64
)


Train on 25000 samples, validate on 25000 samples
Epoch 1/1
22144/25000 [=========================>....] - ETA: 1:16 - loss: 0.4258 - acc: 0.8022

KeyboardInterrupt: 

# BERT pipeline

## Bert tokenizer

In [16]:
# # Create datasets (Only take up to max_seq_length words for memory)
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_df['polarity'].tolist()

test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:max_seq_length]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['polarity'].tolist()

In [17]:
from bert_tokenizer import BERT_tokenizer

In [18]:
tokenizer = BERT_tokenizer(bert_path,max_seq_length)

In [19]:
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = tokenizer.predict(train_text,train_label)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = tokenizer.predict(test_text,test_label)


## Bert model

In [20]:
from bert_layer import BertLayer

In [38]:
def build_model_bert(max_seq_length): 
    in_id = Input(shape=(max_seq_length,), name="input_ids")
    in_mask = Input(shape=(max_seq_length,), name="input_masks")
    in_segment = Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=2, tf_hub = bert_path, output_representation = 'mean_pooling', trainable = True)(bert_inputs)
    
    dense = Dense(256, activation='relu')(bert_output)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    
    return model

In [39]:
bert_model = build_model_bert(max_seq_length)
initialize_vars(sess)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 256)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
bert_layer_6 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [40]:
bert_model.trainable_weights

[<tf.Variable 'bert_layer_6_module/bert/encoder/layer_11/output/LayerNorm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert_layer_6_module/bert/encoder/layer_11/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'dense_11/kernel:0' shape=(768, 256) dtype=float32_ref>,
 <tf.Variable 'dense_11/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_12/kernel:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dense_12/bias:0' shape=(1,) dtype=float32_ref>]

In [ ]:
bert_model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=1,
    batch_size=64
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
22656/25000 [==========================>...] - ETA: 32s - loss: 0.3669 - acc: 0.8377